In [14]:
import numpy as np
import os
import sys
import pandas as pd

import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import pylab as pl
import h5py

import tensorflow as tf
from sklearn.model_selection import ParameterGrid
from itertools import chain
"""
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['CUDA_VISIBLE_DEVICES']="0" 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import ParameterGrid
import re
from sklearn.metrics import confusion_matrix


classes_dir = 'D:\Thesis_ssd\MasterThesis3.0'
os.chdir(classes_dir)
from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.HelperFunctions import HelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.TimeAugmentor import TimeAugmentor
from Classes.Modeling.DynamicModels import DynamicModels
from Classes.Modeling.StaticModels import StaticModels
from Classes.Modeling.NarrowSearchRam import NarrowSearchRam
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Modeling.ResultFitter import ResultFitter
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
import json
#from Classes import Tf_shutup
#Tf_shutup.Tf_shutup()

from livelossplot import PlotLossesKeras



from matplotlib.colors import ListedColormap

plt.rcParams["figure.figsize"]= (15,15)
helper = HelperFunctions()

import sys
ISCOLAB = 'google.colab' in sys.modules
"""
import random
import pprint

base_dir = 'D:\Thesis_ssd\MasterThesis3.0'
os.chdir(base_dir)

In [7]:
main_grid = {
            "num_layers" : [2],
            "batch_size" : [256],
            "epochs" : [40],
            "learning_rate" : [0.01],
            "optimizer" : ["sgd"],
            "start_neurons" : [4],
            "decay_sequence" : [[1,2,4,6,8,10]],
            "dropout_rate" : [0.3],
            "filters" : [17],
            "kernel_size" : [5],
            "padding" : ["same"],
            "use_layerwise_dropout_batchnorm" : [True],
            "l2_r" : [0.001],
            "l1_r" : [0.0001],
            "activation" : ["tanh"],
            "output_layer_activation" : ["sigmoid"]
           }

hyper_grid = {
        "num_layers" : [3,4,5,6],
        "batch_size" : [128, 512, 1028],
        "epochs" : [40],
        "learning_rate" : [0.1, 0.05, 0.025, 0.005],
        "optimizer" : ["sgd"]
    }
model_grid = {
    "start_neurons" : [2,4,6,7],
    "use_layerwise_dropout_batchnorm" : [False, True],
    "decay_sequence" : [[1,2,4,4,2,1], [1,4,8,8,4,1], [1,0.5,0.25,0.25,0.5,1], [1,1,1,1,1,1]],
    "dropout_rate" : [0.3, 0.25, 0.2, 0.15, 0.1],
    "filters" : [11],
    "kernel_size" : [5],
    "padding" : ["same"],
    "l2_r" : [0.005, 0.001, 0.0005, 0.0001, 0],
    "l1_r" : [0.0005, 0.0001, 0.00005],
    "activation" : ["tanh"],
    "output_layer_activation" : ["sigmoid"]
}


model_nr = "LSTM"
is_lstm = True
num_channels = 3

use_time_augmentor = True
use_scaler = True
use_noise_augmentor = True
detrend = False
use_minmax = False
use_highpass = False
highpass_freq = 0.1

use_tensorboard = False
use_liveplots = True
use_custom_callback = False
use_early_stopping = True
start_from_scratch = True


In [28]:
from sklearn.model_selection import ParameterGrid
from collections import OrderedDict

def create_search_space(main_grid, hyper_grid, model_grid, is_dynamic = True):
    hypermodel_grid = {**hyper_grid, **model_grid}
    if not is_dynamic:
        if 'num_layers' in main_grid:
            del main_grid['num_layers']
        if 'decay_sequence' in main_grid:
            del main_grid['decay_sequence']
    key_list = list(main_grid.keys())
    np.random.shuffle(key_list)
    search_list = []
    for key in key_list:
        if len(hypermodel_grid[key]) > 1:
            one_model = main_grid.copy()
            one_model[key] = hypermodel_grid[key]
            print(key, one_model[key])
            print(one_model)
            key_grid = list(ParameterGrid(one_model))
            search_list.append(key_grid)
        else:
            continue
    search_list = list(chain.from_iterable(search_list))
    pprint.pprint(search_list)
    hyper_search, model_search = unmerge_search_space(search_list, hyper_grid, model_grid)
    return hyper_search, model_search

def unmerge_search_space(search_space, hyper_grid, model_grid):
    hyper_keys = list(hyper_grid.keys())
    model_keys = list(model_grid.keys())
    hyper_search_grid = []
    model_search_grid = []
    print(remove_duplicates(search_space))
    pass
    for space in search_space:
        hyper_search_grid.append({key:value for (key,value) in space.items() if key in hyper_keys})
        model_search_grid.append({key:value for (key,value) in space.items() if key in model_keys})
    return hyper_search_grid, model_search_grid

def remove_duplicates(search_space):
    return OrderedDict((frozenset(item.items()),item) for item in search_space).values()


In [29]:
create_search_space(main_grid, hyper_grid, model_grid, is_dynamic = True)

decay_sequence [[1, 2, 4, 4, 2, 1], [1, 4, 8, 8, 4, 1], [1, 0.5, 0.25, 0.25, 0.5, 1], [1, 1, 1, 1, 1, 1]]
{'num_layers': [2], 'batch_size': [256], 'epochs': [40], 'learning_rate': [0.01], 'optimizer': ['sgd'], 'start_neurons': [4], 'decay_sequence': [[1, 2, 4, 4, 2, 1], [1, 4, 8, 8, 4, 1], [1, 0.5, 0.25, 0.25, 0.5, 1], [1, 1, 1, 1, 1, 1]], 'dropout_rate': [0.3], 'filters': [17], 'kernel_size': [5], 'padding': ['same'], 'use_layerwise_dropout_batchnorm': [True], 'l2_r': [0.001], 'l1_r': [0.0001], 'activation': ['tanh'], 'output_layer_activation': ['sigmoid']}
l2_r [0.005, 0.001, 0.0005, 0.0001, 0]
{'num_layers': [2], 'batch_size': [256], 'epochs': [40], 'learning_rate': [0.01], 'optimizer': ['sgd'], 'start_neurons': [4], 'decay_sequence': [[1, 2, 4, 6, 8, 10]], 'dropout_rate': [0.3], 'filters': [17], 'kernel_size': [5], 'padding': ['same'], 'use_layerwise_dropout_batchnorm': [True], 'l2_r': [0.005, 0.001, 0.0005, 0.0001, 0], 'l1_r': [0.0001], 'activation': ['tanh'], 'output_layer_activa

TypeError: unhashable type: 'list'